<table align="left">
  <tr>
    <td><img src="Museum.png" alt="ice cream truck in front of Louvre" width="120"/></td>
    <td align="left"><h1>Lesson 5: Building a Deep Research-like agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Set up and test search tools

In [1]:
from tavily import TavilyClient
from smolagents import tool, Tool
import os
from dotenv import load_dotenv

load_dotenv(override=True) # load variables from local .env file

@tool
def web_search(query: str) -> str:
    """Searches the web for your query.

    Args:
        query: Your query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(query)
    return str(response["results"])


class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, 40000)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

print(web_search("What is the melting temperature of vanilla ice cream in °C?"))

[{'url': 'https://www.quora.com/At-what-temperature-does-ice-cream-melt-Is-it-higher-or-lower-than-water', 'title': 'At what temperature does ice cream melt? Is it higher or lower than ...', 'content': 'Ice Cream will begin melting once when warmer than -15˚C. Shops generally store ice cream in their display freezers at -18˚C, whereas most', 'score': 0.6077801, 'raw_content': None}, {'url': 'https://www.icecreamscience.com/blog/ice-cream-melt', 'title': 'Why does ice cream melt?', 'content': 'Crystallisation of fat globules takes place during the ageing stage where the ice cream mix is kept at 4°C (39.2°F) for at least 4 hours after', 'score': 0.47168615, 'raw_content': None}, {'url': 'https://www.reddit.com/r/AskCulinary/comments/sqaw5h/how_cold_does_ice_cream_really_need_to_be/', 'title': 'How cold does ice cream really need to be? : r/AskCulinary - Reddit', 'content': 'Ice cream is usually best kept around - 10 to - 18 Celsius. Your fridge sounds like one of the older ones so the ic

## Set up an agent

In [6]:
from PIL import Image

from smolagents import CodeAgent, OpenAIServerModel

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

code_model = "gpt-4.1-mini"

model = OpenAIServerModel(
    #"gpt-4o",
    code_model,  #updated model
    max_completion_tokens=8096,
)

<div style="font-size: 20px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> In the notebook, we reduced the museum count from 10 to 3 to reduce runtime and cost.  
  This will reduce the number of ice cream trucks you will need to buy to start your business!
</div>


In [7]:
##changed to 2024 to avoid decaying internet content
#request_museums = """
#Could you give me a sorted list of the top 3 museums in the world in 2024,
#long with their visitor count (in millions) that year, and the approximate daily temperature
#in July at their location ?"""

### Further reduce to 2 museums !!!

In [8]:
#changed to 2024 to avoid decaying internet content
request_museums = """
Could you give me a sorted list of the top 2 museums in the world in 2024,
along with their visitor count (in millions) that year, and the approximate daily temperature
in July at their location ?"""

In [9]:
agent = CodeAgent(
    model=model,
    tools=[web_search, VisitWebpageTool()],
    max_steps=10
)
agent.logger.console.width=66
result = agent.run(request_museums)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 2 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  top_museums_2024 = web_search(query="top museums in the world   
  by visitor count 2024")                                         
  print(top_museums_2024)                                         
 ────────────────────────────────────────────────────────────────

Execution logs:
[{'url': 
'https://www.wanderlustmagazine.com/news/most-visited-museums-2024
/', 'title': 'These were the most-visited museums around the world
in 2024', 'content': 'These were the most-visited museums around 
the world in 2024 - Wanderlust [![Image 
10](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_CultureHeritage.svg)](https://www.wanderlustmagaz
ine.com/inspiration/culture-heritage/)[Culture & 
Heritage](https://www.wanderlustmagazine.com/inspiration/culture-h
eritage/) [![Image 
11](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SustainableTravel.svg)](https://www.wanderlustmag
azine.com/inspiration/sustainable-travel/)[Sustainable 
Travel](https://www.wanderlustmagazine.com/inspiration/sustainable
-travel/) [![Image 
12](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Rail.svg)](https://www.wanderlustmagazine.com/ins
piration/rail/)[Rail](https://www.wanderlustmagazine.com/inspirati
on/rail/) [![Image 
13](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SoloTravel.svg)](https://www.wanderlustmagazine.c
om/inspiration/solo-travel/)[Solo 
Travel](https://www.wanderlustmagazine.com/inspiration/solo-travel
/) [![Image 
14](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_OutdoorsWalking.svg)](https://www.wanderlustmagaz
ine.com/inspiration/outdoors-walking/)[Outdoors & 
Walking](https://www.wanderlustmagazine.com/inspiration/outdoors-w
alking/) [![Image 
15](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Sleeps.svg)](https://www.wanderlustmagazine.com/i
nspiration/sleeps/)[Sleeps](https://www.wanderlustmagazine.com/ins
piration/sleeps/) [![Image 
16](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_NatureWildlife.svg)](https://www.wanderlustmagazi
ne.com/inspiration/nature-wildlife/)[Nature & 
Wildlife](https://www.wanderlustmagazine.com/inspiration/nature-wi
ldlife/) [![Image 
17](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_FoodDrink.svg)](https://www.wanderlustmagazine.co
m/inspiration/food-drink/)[Food & 
Drink](https://www.wanderlustmagazine.com/inspiration/food-drink/)
[![Image 
18](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Trips.svg)](https://www.wanderlustmagazine.com/in
spiration/trips/)[Trips](https://www.wanderlustmagazine.com/inspir
ation/trips/) [![Image 
19](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_PromotedJourneys.svg)](https://www.wanderlustmaga
zine.com/inspiration/promoted-journeys/)[Promoted 
Journeys](https://www.wanderlustmagazine.com/inspiration/promoted-
journeys/) [![Image 
28](https://www.wanderlustmagazine.com/wp-content/uploads/2025/04/
WL238-Cover-1-151x209.jpg)](https://www.wanderlustmagazine.com/mag
azine/april-may-2025/)[April/May 
2025](https://www.wanderlustmagazine.com/magazine/april-may-2025/)
[View 
issue](https://www.wanderlustmagazine.com/magazine/april-may-2025/
) [![Image 
30](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_CultureHeritage.svg)](https://www.wanderlustmagaz
ine.com/inspiration/culture-heritage/)[Culture & 
Heritage](https://www.wanderlustmagazine.com/inspiration/culture-h
eritage/) [![Image 
31](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SustainableTravel.svg)](https://www.wanderlustmag
azine.com/inspiration/sustainable-travel/)[Sustainable 
Travel](https://www.wanderlustmagazine.com/inspiration/sustainable
-travel/) [![Image 
32](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Rail.svg)](https://www.wanderlustmagazine.com/ins
piration/rail/)[Rail](https://www.wanderlustmagazine.com/inspirati
on/rail/) [![Image 
33](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SoloTravel.svg)](https://www.w

[Step 1: Duration 4.99 seconds| Input tokens: 2,010 | Output 
tokens: 120]

...

[Step 3: Duration 6.81 seconds| Input tokens: 12,240 | Output 
tokens: 427]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  # Assign approximate daily July temperatures                    
  top_museums_info[0]["avg_daily_temp_july_f"] = 75               
  top_museums_info[1]["avg_daily_temp_july_f"] = 85               
                                                                  
  # Sort by visitor count descending (already sorted but          
  double-check)                                                   
  sorted_museums = sorted(top_museums_info, key=lambda x:         
  x["visitors_millions"], reverse=True)                           
                                                                  
  final_answer(sorted_museums)                                    
 ────────────────────────────────────────────────────────────────

Out - Final answer: [{'name': 'Louvre Museum', 'city': 'Paris', 
'visitors_millions': 8.7, 'avg_daily_temp_july_f': 75}, {'name': 
'Metropolitan Museum of Art', 'city': 'New York City', 
'visitors_millions': 5.7, 'avg_daily_temp_july_f': 85}]

[Step 4: Duration 3.27 seconds| Input tokens: 19,417 | Output 
tokens: 597]

In [10]:
#modified code to handle case where agent doesn't return a python list.
import pandas as pd

try:
    display(pd.DataFrame(result))
except Exception as e:
    print("Could not display as DataFrame:", e)
    print(result)

,name,city,visitors_millions,avg_daily_temp_july_f
0,Louvre Museum,Paris,8.7,75
1,Metropolitan Museum of Art,New York City,5.7,85


## Create a multi-agent researcher
### Start with web search agent

In [11]:
web_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,  # updated model
        max_completion_tokens=8096,
    ),
    tools=[web_search, VisitWebpageTool()],
    max_steps=10,
    name="web_agent",
    description="Runs web searches for you."
)
web_agent.logger.console.width=66

### Setup manager agent

In [12]:
from smolagents import Tool
from typing import Any
from smolagents.utils import make_image_url, encode_image_base64

def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel(
        "gpt-4o",
    )
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
        "Also, any run that invents numbers should fail."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,
        max_tokens=8096,
    ),
    tools=[],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "plotly.express",  #added
        "plotly.express.colors",  #added
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)
manager_agent.logger.console.width=66

In [13]:
manager_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: ['geopandas', 'plotly', 
│   'plotly.express', 'plotly.express.colors', 'shapely', 'json', 
│   'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description          ┃ Arguments            ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final     │ answer (`any`): The  │
│   │              │ answer to the given  │ final answer to the  │
│   │              │ problem.             │ problem              │
│   └──────────────┴──────────────────────┴──────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description      ┃ Arguments       ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Searches the web │ query           │
            │               │ for your query.  │ (`string`):     │
            │               │                  │ Your query      │
            │ visit_webpage │ Visits a webpage │ url (`string`): │
            │               │ at the given url │ The url of the  │
            │               │ and reads its    │ webpage to      │
            │               │ content as a     │ visit.          │
            │               │ markdown string. │                 │
            │               │ Use this to      │                 │
            │               │ browse webpages. │                 │
            │ final_answer  │ Provides a final │ answer (`any`): │
            │               │ answer to the    │ The final       │
            │               │ given problem.   │ answer to the   │
            │               │                  │ problem         │
            └───────────────┴──────────────────┴─────────────────┘

In [14]:
# added this code, remove plot file if exits.
import os
os.path.exists("saved_map.png") and os.remove("saved_map.png")

In [15]:
manager_agent.run(f"""
{request_museums}

Then make me a spatial map of the world using px.scatter_map, with the biggest museums 
represented as scatter points of size depending on visitor count and color depending 
on the average temperature in July.
Save the map to saved_map.png, then return it!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour",
     color_continuous_scale=px.colors.sequential.Magma_r, size_max=15, zoom=1)
fig.show()
final_answer(fig)

Do not invent any numbers! You must only use numbers sourced from the internet.
""")

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 2 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
│ Then make me a spatial map of the world using px.scatter_map,  │
│ with the biggest museums                                       │
│ represented as scatter points of size depending on visitor     │
│ count and color depending                                      │
│ on the average temperature in July.                            │
│ Save the map to saved_map.png, then return it!                 │
│                                                                │
│ Here's an example of how to plot and return a map:             │
│ import plotly.express as px                                    │
│ df = px.data.carshare()                                        │
│ fig = px.scatter_map(df, lat="centroid_lat",                   │
│ lon="centroid_lon", text="name", color="peak_hour",            │
│      color_continuous_scale=px.colors.sequential.Magma_r,      │
│ size_max=15, zoom=1)                                           │
│ fig.show()                                                     │
│ final_answer(fig)                                              │
│                                                                │
│ Do not invent any numbers! You must only use numbers sourced   │
│ from the internet.                                             │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

────────────────────────── Initial plan ──────────────────────────
Here are the facts I know and the plan of action that I will 
follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task  
- We need the top 2 museums in the world in 2024, ranked by 
visitor count.  
- For each museum: the visitor count in millions for the year 
2024.  
- For each museum: the approximate daily temperature in July at 
the museum’s location.  
- Need to create a spatial map with scatter points:  
  - Size of points depends on visitor count.  
  - Color of points depends on average July temperature.  
- Output the map to a file named saved_map.png.  
- Plotting tool example is plotly.express scatter_map.  
- No invented numbers allowed; must use only sourced accurate 
data.  

### 1.2. Facts to look up  
- Identify the top 2 museums worldwide by visitor count in 2024.  
  - Source: Trusted cultural/museum statistics websites, e.g., The
Art Newspaper Museum Attendance Reports, UNESCO, official museum 
websites, or authoritative news sources.  
- Visitor count (in millions) for each museum for 2024.  
  - Ideally from the same or recent annual reports or aggregate 
data for 2024 year. If 2024 data is not fully available, may need 
to find latest or projected data.  
- Geographic coordinates (latitude, longitude) of each museum.  
  - Source: Official museum websites, Google Maps, Wikipedia.  
- Average daily temperature in July at the museum’s location 
(approximate).  
  - Source: Weather/climate databases such as NOAA, Weatherbase, 
or reputable climate summary sites for the city/location of 
museums.  
- Confirm current year is 2024 and ensure data corresponds to 2024
or projected 2024 values.  

### 1.3. Facts to derive  
- Determine and rank the visitor counts to identify the top 2 
museums.  
- From the location coordinates, extract or verify latitude and 
longitude for plotting.  
- Convert visitor counts into appropriate sizes for scatter plot 
(ex: proportional scaling).  
- Map average July temperature values to a suitable color scale 
for visualization.  
- Compile the data into a DataFrame ready for plotting.  
- Build the scatter_map plot with correct parameters (lat, lon, 
size, color), with proper labels and aesthetics.  
- Save the finished plot as an image named saved_map.png.  

## 2. Plan

1. Conduct web research to find the top museums worldwide by 
visitor count for 2024, including the actual visitor numbers in 
millions.  
2. For each of these two museums, retrieve or confirm their 
geographic coordinates (latitude and longitude).  
3. For each museum’s location, find the average daily temperature 
in July 2024 (or the best available recent average).  
4. Organize the data (museum name, visitor count, latitude, 
longitude, average July temperature) into a DataFrame suitable for
plotting.  
5. Use plotly.express scatter_map to create a spatial map:  
   - Museum locations as points with size proportional to visitor 
count.  
   - Color points based on July temperature, using an appropriate 
continuous color scale.  
   - Add text labels with museum names and visitor counts.  
6. Save the generated plot to saved_map.png.  
7. Return the plot figure as the final answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will start by asking the web agent to find the top 2   
museums in the world for 2024 in terms of visitor count, along    
with their visitor numbers. This will help me gather the most     
relevant and updated data and proceed smoothly from there.        
                                                                  
<code>                                                            
task_description = """                                            
Please find the top 2 museums worldwide by visitor count in 2024. 
Provide the names of these museums and their visitor counts in    
millions for the year 2024.                                       
"""                                                               
top_museums_info = web_agent(task=task_description,               
additional_args={})                                               
print(top_museums_info)                                           
</code>                                                           

─ Executing parsed code: ─────────────────────────────────────── 
  task_description = """                                          
  Please find the top 2 museums worldwide by visitor count in     
  2024.                                                           
  Provide the names of these museums and their visitor counts in  
  millions for the year 2024.                                     
  """                                                             
  top_museums_info = web_agent(task=task_description,             
  additional_args={})                                             
  print(top_museums_info)                                         
 ────────────────────────────────────────────────────────────────

...

Code parsing failed on line 1 due to: SyntaxError
tags with no additional commentary outside, ensuring it is valid 
Python code and does not cause parsing errors.
      ^
Error: invalid syntax (<unknown>, line 1)

[Step 15: Duration 2.71 seconds| Input tokens: 119,427 | Output 
tokens: 2,660]

Reached max steps.

[Step 16: Duration 5.85 seconds| Input tokens: 129,880 | Output 
tokens: 2,973]

'Thought: I will provide the required sorted list and create the spatial map as requested, then return the figure using px.scatter_map with the correct arguments and color continuous scale specified by string name to avoid import errors. I will use verified data from the internet as previously found.\n\n<code>\nimport pandas as pd\nimport plotly.express as px\n\n# Data for top 2 museums in 2024\ndata = {\n    "Museum": ["Louvre Museum", "Vatican Museum"],\n    "VisitorCountMillions": [8.7, 6.8],\n    "Latitude": [48.8606, 41.9065],\n    "Longitude": [2.3376, 12.4536],\n    "JulyTempC": [26.0, 31.1]\n}\n\n# Create DataFrame\ndf = pd.DataFrame(data)\n\n# Sort by visitor count descending\ndf = df.sort_values("VisitorCountMillions", ascending=False)\n\n# Create scatter map plot\nfig = px.scatter_map(\n    df,\n    lat="Latitude",\n    lon="Longitude",\n    size="VisitorCountMillions",\n    color="JulyTempC",\n    color_continuous_scale="Viridis",\n    size_max=40,\n    zoom=1,\n    text="M

> Note, the map will look a bit different as we are only searching for 3 locations.

In [25]:
import sys
print(f"Installing nbformat into: {sys.prefix}")
!uv pip install "nbformat>=4.2.0"

Installing nbformat into: /Users/ytchen/Documents/experimental/CodeAgent/.venv
Using Python 3.11.13 environment at: /Users/ytchen/Documents/experimental/CodeAgent/.venv
Audited 1 package in 5ms


In [27]:
import sys
print(f"Current Python executable: {sys.executable}")
print(f"Python path: {sys.path}")

try:
    import nbformat
    print(f"nbformat is installed. Version: {nbformat.__version__}")
except ImportError:
    print("nbformat is NOT installed in this environment.")

Current Python executable: /Users/ytchen/Documents/experimental/CodeAgent/.venv/bin/python
Python path: ['/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/ytchen/Documents/experimental/CodeAgent/.venv/lib/python3.11/site-packages']
nbformat is installed. Version: 5.10.4


### It didn't work, somehow couldn't find "nbformat>=4.2.0" even it already installed.
### The alternative is better as follows.

In [29]:
"""
fig = manager_agent.python_executor.state["fig"]
fig.update_layout(width=700, height=700)
fig.show()
"""

'\nfig = manager_agent.python_executor.state["fig"]\nfig.update_layout(width=700, height=700)\nfig.show()\n'

In [28]:
from IPython.display import HTML
import plotly.io as pio

# Assuming 'fig' is your Plotly figure object
fig = manager_agent.python_executor.state["fig"]
fig.update_layout(width=700, height=700)

# Convert the figure to an HTML string
html_str = pio.to_html(fig, full_html=False, include_plotlyjs='cdn')

# Display the HTML in the Jupyter cell
display(HTML(html_str))